In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pprint import pprint
from dotenv import load_dotenv
from tqdm.autonotebook import tqdm
from pathlib import Path

from glob import glob

from pinecone import Pinecone

from pinecone_text.sparse import BM25Encoder
import nltk

root_dir = Path(os.getcwd()).parent.parent
sys.path.insert(0, str(root_dir))

C:\Users\edgarmp\AppData\Local\Temp\ipykernel_27720\411096512.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# from src.d00_utils.parsing import extract_index_from_doc
from src.d01_data.data import read_json, create_index_if_not_exists, upsert_vectors_in_batches
from src.d00_utils.utils import dict_to_document, metadata_to_uuid, generate_sparse_vector_in_batches

nltk.download('punkt_tab')

load_dotenv('../../.env')
 
raw_path = root_dir / 'data' / '01_raw'
intermediate_path = root_dir / 'data' / '02_intermediate'
output_path = root_dir / 'data' / '04_model_output'

files_path = [Path(p) for p in glob(str(intermediate_path / '*.json'))]

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\edgarmp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
pc = Pinecone()
pc_index = create_index_if_not_exists(client=pc, index_name='chatbot-leyes')

The index chatbot-leyes already exists


In [ ]:
# Converting the json to Documents object with its content and metadata
all_articles = []
for file in files_path:
    articles = read_json(file)
    all_articles.extend(dict_to_document(estructured_dict=articles, origen=file.stem))
    
print(f'{len(all_articles)} documents have been created')

2774 documents have been created


In [ ]:
# Checks for duplicates chunks
final_uuids = metadata_to_uuid(all_articles)

assert len(final_uuids) == len(set(final_uuids)), 'THE ARE DUPLICATED IDS'
del final_uuids

In [ ]:
# Training and saving the BM25Encoder
bm25 = BM25Encoder(language='spanish')
bm25.fit([doc.page_content for doc in all_articles])
bm25.dump(output_path / 'bm25_values.json')

  0%|          | 0/2774 [00:00<?, ?it/s]

In [37]:
print('Creating embeddings')
vectors = generate_sparse_vector_in_batches(documents=all_articles,
                                            embedding_client=pc.inference,
                                            fitted_bm25=bm25,
                                            batch_size=64)
print('Upserting chunks ')
upsert_vectors_in_batches(vectors=vectors, index=pc_index, batch_size=100)

Upserting chunks 


100%|██████████| 28/28 [01:19<00:00,  2.83s/it]
